<a href="https://colab.research.google.com/github/SiddSen/ap_research_research/blob/main/TF_slim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tf_slim
!pip install roboflow

In [ ]:
!git clone https://github.com/tensorflow/models/

In [ ]:
%cd models/research/slim

In [ ]:
%mkdir dataset

In [ ]:
DOWNLOAD_COMMAND = "curl -L \"https://app.roboflow.com/ds/7WiwxYZewU?key=8szFkSB8H7\" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip" #@param {type:"string"}
!cd dataset && {DOWNLOAD_COMMAND} %% cd .. 


In [ ]:
import tf_slim as slim
eval = slim.evaluation.evaluate_once

In [ ]:
from converter import _convert_dataset, _get_filenames_and_classes

photos, ids = _get_filenames_and_classes('/content/train')
_convert_dataset(photos, ids, 'converted_train')

In [ ]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
r"""Downloads and converts Flowers data to TFRecords of TF-Example protos.

This module downloads the Flowers data, uncompresses it, reads the files
that make up the Flowers data and creates two TFRecord datasets: one for train
and one for test. Each TFRecord dataset is comprised of a set of TF-Example
protocol buffers, each of which contain a single image and label.

The script should take about a minute to run.

"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import sys

import tensorflow.compat.v1 as tf

# The number of shards per dataset split.
_NUM_SHARDS = 5


class ImageReader(object):
  """Helper class that provides TensorFlow image coding utilities."""

  def __init__(self):
    # Initializes function that decodes RGB JPEG data.
    self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
    self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

  def read_image_dims(self, sess, image_data):
    image = self.decode_jpeg(sess, image_data)
    return image.shape[0], image.shape[1]

  def decode_jpeg(self, sess, image_data):
    image = sess.run(self._decode_jpeg,
                     feed_dict={self._decode_jpeg_data: image_data})
    assert len(image.shape) == 3
    assert image.shape[2] == 3
    return image


def _get_filenames_and_classes(dataset_dir):
  """Returns a list of filenames and inferred class names.

  Args:
    dataset_dir: A directory containing a set of subdirectories representing
      class names. Each subdirectory should contain PNG or JPG encoded images.

  Returns:
    A list of image file paths, relative to `dataset_dir` and the list of
    subdirectories, representing class names.
  """
  directories = []
  class_names = []
  for filename in os.listdir(dataset_dir):
    path = os.path.join(dataset_dir, filename)
    if os.path.isdir(path):
      directories.append(path)
      class_names.append(filename)

  photo_filenames = []
  for directory in directories:
    for filename in os.listdir(directory):
      path = os.path.join(directory, filename)
      photo_filenames.append(path)

  return photo_filenames, sorted(class_names)


def _get_dataset_filename(dataset_dir, shard_id):
  output_filename = 'mpox_%05d-of-%05d.tfrecord' % (
    shard_id, _NUM_SHARDS)
  return os.path.join(dataset_dir, output_filename)


def _convert_dataset(filenames, class_names_to_ids, dataset_dir):
  """Converts the given filenames to a TFRecord dataset.

  Args:
    filenames: A list of absolute paths to png or jpg images.
    class_names_to_ids: A dictionary from class names (strings) to ids
      (integers).
    dataset_dir: The directory where the converted datasets are stored.
  """
  num_per_shard = int(math.ceil(len(filenames) / float(_NUM_SHARDS)))

  with tf.Graph().as_default():
    image_reader = ImageReader()

    with tf.Session('') as sess:

      for shard_id in range(_NUM_SHARDS):
        output_filename = _get_dataset_filename(
            dataset_dir, shard_id)

        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
          start_ndx = shard_id * num_per_shard
          end_ndx = min((shard_id+1) * num_per_shard, len(filenames))
          for i in range(start_ndx, end_ndx):
            sys.stdout.write('\r>> Converting image %d/%d shard %d' % (
                i+1, len(filenames), shard_id))
            sys.stdout.flush()

            # Read the filename:
            image_data = tf.gfile.GFile(filenames[i], 'rb').read()
            height, width = image_reader.read_image_dims(sess, image_data)

            class_name = os.path.basename(os.path.dirname(filenames[i]))
            class_id = class_names_to_ids[class_name]

            # example = dataset_utils.image_to_tfexample(
            #     image_data, b'jpg', height, width, class_id)
            # tfrecord_writer.write(example.SerializeToString())

  sys.stdout.write('\n')
  sys.stdout.flush()

In [ ]:
Selected_model = "Resnet" #@param ["Resnet", "VGG", "Mobilenet"]

MODEL_LIST = {
    "Resnet" : "http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz",
    "VGG" : "http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz",
    "Mobilenet" : "https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.4_224.tgz"
}

MODEL_NAME = Selected_model
MODEL_LINK = MODEL_LIST[Selected_model]
PRETRAINED_NAME = MODEL_LINK[MODEL_LINK.index(MODEL_NAME[0 : 2].lower()) : MODEL_LINK.index('.tar')]

In [ ]:
!wget {model}
!tar -zxvf {PRETRAINED_NAME + ".tar.gz"}

--2023-01-30 17:59:22--  http://%7Bmodel%7D/
Resolving {model} ({model})... failed: Name or service not known.
wget: unable to resolve host address ‘{model}’
tar (child): resnet_v1_50_2016_08_28.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
!cp dataset/test/*.tfrecord dataset/compiled/test.tfrecord
!cp dataset/train/*.tfrecord dataset/compiled/train.tfrecord
!cp dataset/valid/*.tfrecord dataset/compiled/valid.tfrecord
!cp dataset/train/*.pbtxt dataset/compiled/labels.pbtxt

TEST_FILE = COMPILED_DIR + "/test.tfrecord"
TRAIN_FILE = COMPILED_DIR + "/train.tfrecord"
VALID_FILE = COMPILED_DIR + "/valid.tfrecord"
LABELS_FILE = COMPILED_DIR + "/labels.pbtxt"